In [12]:
from pathlib import Path

import automech
from project_utilities import p_, util, workflow

file = util.notebook_file() if util.is_notebook() else __file__
tag = util.file_tag(file)
root_path = Path("..")
par_mech = workflow.read_parent_mechanism(root_path=root_path)

In [13]:
# Generate submechanism
from automol.graph import enum

gen_mech = automech.from_smiles(
    spc_smis=["C1=C[CH]CC1", "C1=CC[CH]C1"],
    src_mech=par_mech,
)
#  - H-migrations
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.h_migration_12,
    src_mech=par_mech,
)
#  - Ring-opening scissions
gen_mech = automech.enumerate_reactions(
    gen_mech,
    enum.ReactionSmarts.ring_beta_scission,
    src_mech=par_mech,
)
automech.display(gen_mech)

  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# Expand stereochemistry
ste_mech, *_, gen_mech = workflow.expand_stereo(
    mech=gen_mech, tag=tag, root_path=root_path
)


Expanding stereochemistry...


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]


Stereoexpansion errors:


In [15]:
# Prepare calculation
workflow.prepare_calculation(
    gen_mech=gen_mech, ste_mech=ste_mech, tag=tag, root_path=root_path
)


Sorting mechanism...

Writing mechanism...
../data/1e_C_r_v0_gen.json
../data/1e_C_r_v0_ste.json
../data/mechanalyzer/1e_C_r_v0_ste.dat
../data/mechanalyzer/1e_C_r_v0_ste.csv


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [16]:
# # Gather statistics
# workflow.gather_statistics(tag=tag, root_path=root_path)

In [17]:
# # Prepare simulation
# workflow.prepare_simulation(tag=tag, root_path=root_path)
# workflow.prepare_simulation_species(tag=tag, root_path=root_path)

In [18]:
# # Check rates
# workflow.plot_rates(tag=tag, root_path=root_path)

In [19]:
# # Plot simulation
# chart_dct = workflow.plot_simulation(
#     tag=tag,
#     x_col="O2_molecules",
#     root_path=root_path,
#     line_source_=["Hill", "Lokachari"],
#     point_source="Experiment",
#     control=False,
# )
# mech = automech.io.read(
#     p_.full_calculated_mechanism(tag, "json", path=p_.data(root_path))
# )
# for name, chart in chart_dct.items():
#     automech.display_species(mech, [name])
#     chart.show()

In [20]:
# # Check TS
# from mechdriver.subtasks import display

# chan = "2: 5"

# #   - Read in expanded mechanism
# mech_path = p_.mechanism(tag, ext="json", path=p_.data(root_path))
# mech = automech.io.read(mech_path)

# #   - Display the reaction
# automech.display_reactions(mech, chans=[chan])

In [21]:
# #   - Display the TS mode
# calc_path = p_.calc(root_path, tag)
# display("conf_opt", chan, path=calc_path)

In [22]:
# #   - Display the TS mode
# calc_path = p_.calc(root_path, tag)
# display("find_ts", chan, path=calc_path)